# Information Retrieval

# Lesson 3: Master index data structures


## Vector Index in Information Retrieval

A vector index is a crucial data structure in the field of Information Retrieval (IR) that enables efficient querying and retrieval of high-dimensional vectors. It is particularly useful for tasks that involve finding the most relevant or similar items in a large dataset, such as document retrieval, recommendation systems, and semantic search.

### Key Concepts

1. **Embedding**: In IR, embedding refers to the process of converting text or other types of data into fixed-size vector representations. These vectors capture the semantic meaning of the data and are often generated using models like Word2Vec, GloVe, BERT, or FastText.

2. **Nearest Neighbor Search**: This is the task of finding the vectors in the dataset that are closest to a given query vector. Nearest neighbor search is fundamental for IR tasks such as finding similar documents, recommending items, or retrieving relevant information based on a query.

3. **Indexing Methods**: Various algorithms and data structures can be used to build a vector index, each with its own strengths and weaknesses:
    - **KD-Tree**: A binary tree that partitions the space for efficient range and nearest neighbor searches. Suitable for low-dimensional data.
    - **Ball Tree**: Uses hyperspheres to partition the space, which can be more efficient for certain types of data compared to KD-Tree.
    - **Annoy**: Builds a forest of random projection trees for fast approximate nearest neighbor searches. Useful for large-scale datasets.
    - **HNSW (Hierarchical Navigable Small World)**: A graph-based method that provides efficient and scalable nearest neighbor search.
    - **FAISS (Facebook AI Similarity Search)**: A library offering various indexing methods, including Inverted File (IVF) and Product Quantization (PQ), for large-scale similarity search.


## Where is a hospital in Manhattan Downtown?

In this lab we will create 2 vector indices to answer a very simple question: if you are in Manhattan downtown, where is the nearest hospital? We will base our soultion on two sources of data:
- [Points of Interest dataset](https://drive.google.com/file/d/1LUudtCADqSxRl18ZzCzyPPGfhuUo2ZZs/view?usp=sharing). This is a 10% sample of a bigger dataset. Download and uncompress the file.
- [Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/start) or any other [equivalent service](https://gisgeography.com/geocoders/). For Google you will need to obtain a key. **PLEASE DO NOT SUBMIT THE KEY :)**


## To complete this lab we will go through several steps:
1. Build **coordinate search index**. We will use it to obtain POI from the given region.
3. Implement **vector text embedding index** (Annoy, HNSW) to serve semantic queries.
3. Implement **geocoding** with cache. We will use it to obtain city coordinates.
4. Impement search for **double queries: town and location type**.

##  0. Prepare data

In [ ]:
import matplotlib.pyplot
import pandas as pd
import json

def draw_earth(xlim=(-180, +180), ylim=(-90, +90)):
    plt.figure(figsize=(15, 8))
    plt.xlim(xlim)
    plt.ylim(ylim)

    # this file also lives in github. Adjust the path if needed.
    df = pd.read_csv("./world.csv")

    for row in df['geojson']:
        js = json.loads(row)
        polys = js['coordinates']
        for poly in polys:
            for pp in poly:
                x, y = [v[0] for v in pp], [v[1] for v in pp]
                plt.plot(x, y, color='gray')

Reading the dataset and storing coordinates in `GEO` matrix:

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

# replace filename if you want to use another data file
# be careful! 2M points is still a big number and can eat significant amout of memory
with open("poi_sample01.pickle", "rb") as f:
    dataset = pickle.load(f)

# let's sample 20000 of points to draw
step = len(dataset) // 20000

# pure coordinated in compressed representation, 2B per number -> 8MB per array
GEO = np.array([v[0] for v in dataset], dtype=np.float16)
N = len(dataset)
# free the memory
dataset = None
import gc
gc.collect()

Showing approximate dataset data distribution:

In [ ]:
draw_earth(ylim=(-75, 75))
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.grid()
plt.scatter(GEO[::step, 0], GEO[::step, 1])
plt.show()

## 0.1. Read the data from the hard drive

We will prepare id-based shards (data will be distributed into equal files with ranges `[0..capacity-1], [capacity..2*capacity-1], ...`. Each shard will store `capacity` elements. Your task is to complete the implementation with `iterate_dataset` function.

In [ ]:
def split_shards(file, folder='shard', capacity=20000):
    import pickle, os, math, gc
    if not os.path.exists(folder):
        os.mkdir(folder)
    with open(file, "rb") as f:
        dataset = pickle.load(f)
    nshards = len(dataset) // capacity
    if nshards * capacity < len(dataset):
        nshards += 1

    for i in range(nshards):
        with open(f"{folder}/{i}", 'wb') as f:
            part = dataset[i * capacity:(i+1)*capacity]
            pickle.dump(part, f)
    dataset = None
    gc.collect()


def dataset_get(indices, folder='shard', capacity=20000) -> list:
    result = []
    groups = {}
    for i in indices:
        x = i // capacity
        if x not in groups:
            groups[x] = []
        groups[x].append(i)
    for x in groups:
        with open(f"{folder}/{x}", "rb") as f:
            sha = pickle.load(f)
            for i in groups[x]:
                row = sha[i % capacity]
                result.append(row)
    return result


# should return iterator, which goes through all elements, consequently opening files
# use ``yield`` operator to simplify your code
def iterate_dataset(items, folder="shard", capacity=20000):
    # TODO write your code instead
    yield ([0.0, 0.0], "stub")

In [ ]:
split_shards("poi_sample01.pickle")

### 0.1.1. Tests

In [ ]:
import numpy as np

for i in [137, 40000, 600000]:
    assert np.allclose(GEO[i,:], dataset_get([i])[0][0], atol=5*1e-2), ""

In [ ]:
dataset_get([1, 10, 1000234, N-1])

# 1. Create spacial index for POI (points of interest)

We will store dataset rows numbers as values, and coordinates as keys. Please use [KDtree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree) or [BallTree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree) from sklearn.

## 1.1. Build the index and return it

Implement the following functions:
- `build_geospacial_index` should build and return a search tree object: KDTree or BallTree.
- `kNN` accepts a 2D-point, `k` neighbours parameter, and returns **approximate** `k` neighbours (they can be different from the real neighbours).
- `inRadius` accepts a 2D-point, L<sub>2</sub> `radius`, and returns points inside the radius. Clarification: for simplicity **radius is given in units of coordinates (degrees)**, not kilometers or meters.


### BallTree
BallTree is a data structure used for efficient nearest neighbor searches. It partitions the data space using hyperspheres (balls) to create a tree structure. Each node in the tree represents a ball that contains a subset of the data points. This method is particularly effective for high-dimensional data and can handle various distance metrics.

![](https://miro.medium.com/v2/resize:fit:700/1*bdt854gdCrd_kKQECAELPA.png)

### KDTree
KDTree (K-Dimensional Tree) is another data structure for efficient nearest neighbor searches. It partitions the data space using hyperplanes, creating a binary tree where each node represents a split along one of the data dimensions. KDTree is well-suited for low to moderate-dimensional data and typically uses Euclidean distance for its operations.

![](https://miro.medium.com/v2/resize:fit:700/1*6LuJXjGifBSm7lEEO5sftQ.png)

In [ ]:
from sklearn.neighbors import BallTree

def build_geospacial_index(points, leaf_size=5) -> BallTree: # or KDTree here
    #TODO write your code here
    tree = None
    return tree


def kNN(query_point: list, k: int, index: BallTree) -> list:
    # TODO your code here
    pass


def inRadius(query_point: list, r: float, index: BallTree) -> list:
    # TODO your code here
    pass

In [ ]:
spaidx = build_geospacial_index(GEO)

### 1.1.1. Tests

In [ ]:
test_id = 13

idx = kNN(GEO[test_id], 10, spaidx)
print(sorted(idx))
assert test_id in idx, "Point itself should be in results"

idx = inRadius(GEO[test_id], 0.0625, spaidx)
print(sorted(idx))
assert test_id in idx, "Point itself should be in results"

## 1.2. Tricky assert

Some keys (coordinates) in the dataset (surprise!) are duplicates. Unfortunately search trees (in basic implemenation) cannot support duplicates. Thus you can follow one of the strategies:
- a key (coordinateS) corresponds to multiple values. This may require additional data strictures.
- improve the data (coordinates) to avoid collisions (e.g. make sure they never coinside by adding insignificant noise)

You need to modify `kNN` function from previous step.

### 1.2.1. Tests

In [ ]:
points = [1966663, 1480877, 2126566]
for p in points:
    x = GEO[p, :]
    r = kNN(x, 1000, spaidx)
    assert (p in r), "Query did not return itself"

## 1.3. How leaf size influences build and search speed?

Let us look at how parameter of leaf size affects speed of search and construction.
We will test on sizes: 1, 10, 20 and 50.

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import tqdm

queries = random.sample(range(N), 1000)
leaf_sizes = [1, 10, 20, 50]

build_times = []
query_times = []
for ls in tqdm.tqdm(leaf_sizes):
    start = time.time()
    idx = build_geospacial_index(GEO, ls)
    build_times.append(time.time() - start)

    start = time.time()
    for q in queries:
        d, r = spaidx.query([GEO[q]], 10000, sort_results=False, breadth_first=True)
    query_times.append(time.time() - start)
    idx = None
    gc.collect()

plt.xlabel("build time, s")
plt.ylabel("query time, s")
plt.scatter(build_times, query_times)
for i, ls in enumerate(leaf_sizes):
    plt.annotate(str(ls), (build_times[i], query_times[i]))

## 1.4. Range queries?

Ok, you have a **radius query**, but what about **rectangual ranges**? Using the functions you already wrote, please, implement the range query given `north-east` and `south-west` corners. Pass the asserts to get points.

Hint: The north-east and south-west corners are boundaries of your rectangular range.
Iterate through your dataset and check if each point lies within the defined rectangular boundaries. This involves checking if the point's latitude and longitude fall between the corresponding coordinates of the north-east and south-west corners.

In [ ]:
import numpy as np
from scipy.spatial import distance

# should return ids of the rows in this range
def get_in_range(ne, sw, spacial_index, GEO) -> list:
    # TODO
    return [-1, 0, 23]

In [ ]:
def print_starbucks(ids):
    for row in dataset_get(ids):
        if 'Starbucks' in row[1]:
            print(row[1])

### 1.4.1. Tests

In [ ]:
ids = get_in_range([-73.97, 40.75], [-74.03, 40.70], spaidx, GEO)

assert any(map(
            lambda x: 'Manhattan, 80 Delancey St' in x[1],
            dataset_get(ids))), "This Starbucks should be in place!"

print_starbucks(ids)

# 2. Geocoding

In this block we will learn, how to convert text place names into coordinate rectangles.

**Geocoding** is the process of converting addresses or place names into geographic coordinates, such as latitude and longitude. This is essential for mapping applications and spatial analysis, as it allows us to place locations on a map and perform geographic queries. Geocoding services, like the Google Geocoding API, provide this functionality by taking a textual location description and returning the corresponding coordinates.

## 2.1. Implement geocoding
Complete `get_town_range_coordinates` function which returns north-eastern and south-western points of the place. 

You can use Google Gecoding API or Yandex Geocoder API. Google required credit card validation, so I reccomend to use Yandex

In [ ]:
import requests

# this function returns a pair of tuples: NE and SW corners.
def get_town_range_coordinates(town: str, api_key: str) -> tuple:
    api = "https://geocode-maps.yandex.ru/1.x?geocode={}&apikey={}&format=json"
    # TODO your code here.
    # implement the function that will return the NE and SW corners of the town
    return NE, SW

If needed, request your key here: https://yandex.ru/dev/maps/geocoder/

In [ ]:
my_yandex_geocoder_api_key = open('yandex.key', 'r').read()

### 2.1.1 Tests

In [ ]:
p = get_town_range_coordinates('Pittsburgh downtown', my_yandex_geocoder_api_key)
print(p)
assert p[1][0] <= -80. <= p[0][0] and p[1][1] <= 40.44 <= p[0][1]

## 2.2. Town queries

Now, having a range query and geocoding, we can implement town-queries! Pass the assert to get the points.

In [ ]:
# should return dataset indices
def get_in_town(town, index, GEO, maps_api_key) -> list:
    # TODO your code here!
    return [0, 1, 2]

### 2.2.1. Tests

In [ ]:
ids = get_in_town('Pittsburgh downtown', spaidx, GEO, my_yandex_geocoder_api_key)

assert any(map(
            lambda x: 'US, Pittsburgh, 810 River Ave' in x[1],
            dataset_get(ids))), "This Starbucks should be in place!"

print_starbucks(ids)

## 2.3. Caching

Why should you pay for every geocaching request, if you can cache them? Implement a cached version on geocoding. The second query does not use internet.

**Cache** is a high-speed data storage layer that stores a subset of data, typically transient in nature, so that future requests for that data are served up faster than accessing the primary storage location. Caching allows for the reuse of previously retrieved or computed data, reducing the time and resources needed to access frequently used information. It is commonly used in computing to improve the performance and efficiency of systems by minimizing the need to repeatedly fetch or compute the same data.

In this case Cache could be a dict where key - town name and value - coordinates

In [ ]:
global GEO_CACHE

def get_town_range_coordinates_cached(town: str, maps_key: str) -> tuple:
    global GEO_CACHE
    # TODO your code here
    pass


def get_in_town_cached(town: str, index, GEO, maps_key: str) -> list:
    # TODO your code here
    pass

In [ ]:
ids = get_in_town_cached('Boulder, CO', spaidx, GEO, my_google_maps_api_key)
print_starbucks(ids)
ids = get_in_town_cached('Boulder, CO', spaidx, GEO, my_google_maps_api_key)
print_starbucks(ids)

# 3. Text search

We are done with geography, but we have no clear method to search for categories. What if we prepare vector index of location names?

In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
names = []

## 3.1. Embedding


**Embeddings** are dense vector representations of text or other data types that capture semantic meaning and relationships between items. They are commonly used in NLP and ML to transform high-dimensional data into a lower-dimensional space for efficient processing and analysis. In our case, embeddings help us represent addresses and place names as vectors for efficient similarity search and retrieval.


Here is the trick. If you use any embedding model "as it is", it may take some hours to prepare 2M embeddings. It's ok if you can wait, but...

Please think, how you can speed up the process with embedding to less than 5 minutes?

HINT: spacy model `nlp` has a [dictionary for word embeddings](https://spacy.io/api/vocab). You can access `nlp.vocab[word].vector` to get word embedding, `nlp.vocab.strings` map stores integer indices. 

In [ ]:
from tqdm import tqdm
import gc
import numpy as np

embeddings = np.zeros((N, 300), dtype=np.float16)

for i, item in enumerate(tqdm(iterate_dataset(N), total=N)):
    name = item[1].split('.')[0]
    emb = embed(name, nlp)
    if emb is not None:
        embeddings[i, :] = emb

gc.collect()

## 3.2. Vector index

Here you build vector index for our embeddings. I want to warn Windows users, that they can observe problems with installing Faiss and HNSWlib (please refer to the corresponding lab). Still this is not the reason not to try :)
Choose **one of the libraries** and fulfill the requirements to get full points:
1. If you choose [FAISS](https://faiss.ai/). Get started with [installation](https://faiss.ai/#install) and this [tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started). To get full points your index must use [Product Quantization](https://github.com/facebookresearch/faiss/wiki/Lower-memory-footprint): 50 subvectors, 8 bits (1 byte) each. Use custom `nprobe` parameter equal to 23. Number or Voronoi cells is `65536`. Refer to [this document](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index#if-1m---10m-ivf65536_hnsw32) to understand recommendations.
2. If you use [HNSWlib](https://github.com/nmslib/hnswlib) (or [nmslib](https://github.com/nmslib/nmslib)) then follow these requirements. Use `cosine` metric for index construction, maximum number of outgoing connections (max outdegree) in the graph is 16, `ef` parameter at construction time should be `250`. Some useful information is given [here](https://github.com/nmslib/nmslib/blob/master/manual/methods.md).
3. For [Annoy](https://github.com/spotify/annoy) you should use cosine distance for the space (if vectors are normed, you can use dot product intead), use all CPU cores at construction time. Build the index right on the disk, then load. Your index should consist of 37 trees.

**NB** If you run on not-very-modern hardware (e.g. your RAM is less then 8GB), then you'd better reduce dataset size (e.g. take a specific region only like US east cost). You can also reduce other parameters only for the sake of RAM efficiency, but please specify and justify your decisions.

e.g.
```    
roi = set(get_in_range([-68.645945, 43.163175], [-80.461502, 37.097044], spaidx, GEO))
```


**HINT** You can remove `embeddings` array and call `gc.collect()` before loading index to RAM.

In [ ]:
embedding_index = None

In [ ]:
import numpy as np

def get_vector_index():
     # TODO returns index object
    # Write your code here
    pass
    


def get_kNN_embeddings(embedding, k, index):
    # TODO write your kNN queries here
    pass

In [ ]:
embedding_index = get_vector_index()

### 3.2.1. Tests

In [ ]:
result = get_kNN_embeddings(embed('pharmacy', nlp), 1000, embedding_index)
assert len(result) == 1000

# 4. And now we want to have this together!

Implement `find` function, which takes a town and a location which you want to find there

In [ ]:
def find(town, query) -> list:
    # TODO should return list of data items, with coordinates and texts

In [ ]:
items = find('Manhattan downtown', 'hospital')
print(items[:20])
xy = np.array([row[0] for row in items])

In [ ]:
NE, SW = get_town_range_coordinates_cached('Manhattan downtown', my_yandex_geocoder_api_key)
draw_earth(xlim=(SW[0] - 5, NE[0] + 5), ylim=(SW[1] - 5, NE[1] + 5))
plt.scatter(xy[:, 0], xy[:, 1])
plt.show()